In [1]:
import nltk
import gensim
import numpy as np
import pandas as pd
import re,string,unicodedata

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from gensim.utils import simple_preprocess 
from gensim.models import Phrases, Word2Vec
from sklearn.preprocessing import LabelBinarizer
from nltk.tokenize import word_tokenize,sent_tokenize


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Load the IMDb dataset from the CSV file
imdb_df = pd.read_csv('/imdb_dataset.csv', header=None, names=['review', 'sentiment'])
imdb_df["sentiment"] = np.where(imdb_df["sentiment"] == "positive", 0, 1)

gen_df = pd.read_csv('/GEN-sarc-notsarc.csv', header=None, names=['class', 'text'])
gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))

gen_df["class"] = np.where(gen_df["class"] == "notsarc", 0, 1)

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

# Preprocess the dataset
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

<ipython-input-2-5c5b39dec5c3>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
<ipython-input-2-5c5b39dec5c3>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))


In [3]:
imdb_df['review']=imdb_df['review'].apply(denoise_text)
gen_df['text']=gen_df['text'].apply(denoise_text)

<ipython-input-2-5c5b39dec5c3>:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [4]:
words, imdb_words, gen_words = [], [], []
for i in imdb_df.review.values:
    words.append(i.split())
    imdb_words.append(i.split())
words=words[1:]

for i in gen_df.text.values:
    words.append(i.split())
    gen_words.append(i.split())

In [5]:
imdb_wv_model = Word2Vec(words, vector_size=500, window=3, min_count=1, workers=16)

In [6]:
print(imdb_wv_model)

Word2Vec<vocab=485482, vector_size=500, alpha=0.025>


In [7]:
print(len(words))

59387


In [8]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(485482)
tokenizer.fit_on_texts(words)
text = tokenizer.texts_to_sequences(words)
text = keras.utils.pad_sequences(text, 75)

print(text[:2])

[[   145    166  42448    703     75   1528   4516   2772   7673    703
    7962    703 154647   1293   2081     17    355     37     99   3229
    1655  11805     43   1401     95    159   1888   1562   1551  15623
      79  11649    196   3075   2146   3154   2091  10361 154648   6038
   15624   2956 105798   8723  15624    341    483     10    168     95
      26  54856    604    823   8723    466   1376  33356    478    366
    1051   2583   1376  54857     57  15623     84    247   3879   3815
  154649     10   1237   4495   4324]
 [   201   2519    252   5587     86    369     26 154652    374   6640
   45740     79 154653   2650   3700   5609    203      9  21813    913
   11806   1916  14464  11483  33357     26    148     57  61470    345
    2524   5587   4964    291      3     18  19547    172    389   2519
       8    138    302     27  11650   1119   3719   2270    121    190
    2027 105800   3857   1095   1080  21183    164   1799  18585    419
      64   3416  27819  39

In [9]:
y = imdb_df["sentiment"]

In [10]:
txt = tokenizer.texts_to_sequences(imdb_words)
txt = keras.utils.pad_sequences(txt, 75)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(txt), np.array(y), train_size=0.8, stratify=y)

In [16]:
model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=500,
                            input_length=75,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    keras.layers.Conv1D(50, 3, activation='relu', padding='same', strides=1),
    keras.layers.MaxPool1D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(10),
    keras.layers.Activation('relu'),
    keras.layers.Dense(1),
    keras.layers.Activation('sigmoid')
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 75, 500)           242741000 
                                                                 
 conv1d_13 (Conv1D)          (None, 75, 50)            75050     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 37, 50)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 1850)              0         
                                                                 
 dense_4 (Dense)             (None, 10)                18510     
                                                                 
 activation_4 (Activation)   (None, 10)                0         
                                                      

In [ ]:
val_recall = 0.8160
val_precision = 0.8572
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

0.8360927563949319

In [17]:
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 75)

y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
235/235 [==============================] - 47s 197ms/step - loss: 0.4818 - accuracy: 0.8398 - precision_2: 0.8478 - recall_2: 0.9884 - val_loss: 0.4013 - val_accuracy: 0.8472 - val_precision_2: 0.8472 - val_recall_2: 1.0000
Epoch 2/10
235/235 [==============================] - 26s 112ms/step - loss: 0.2622 - accuracy: 0.8688 - precision_2: 0.8703 - recall_2: 0.9932 - val_loss: 0.5286 - val_accuracy: 0.8360 - val_precision_2: 0.8570 - val_recall_2: 0.9679
Epoch 3/10
235/235 [==============================] - 21s 92ms/step - loss: 0.1140 - accuracy: 0.9543 - precision_2: 0.9655 - recall_2: 0.9811 - val_loss: 0.7867 - val_accuracy: 0.8056 - val_precision_2: 0.8671 - val_recall_2: 0.9101
Epoch 4/10
235/235 [==============================] - 18s 76ms/step - loss: 0.0489 - accuracy: 0.9832 - precision_2: 0.9923 - recall_2: 0.9879 - val_loss: 0.8801 - val_accuracy: 0.7843 - val_precision_2: 0.8660 - val_recall_2: 0.8818
Epoch 5/10
235/235 [==============================] - 18s 77ms

In [18]:
val_recall = 0.8693 *100
val_precision = 0.8567 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

86.29540092699884

In [19]:
model.save("/cnn-imdb.h")

In [20]:
gen_wv_model = Word2Vec(gen_words, vector_size=500, window=3, min_count=1, workers=16)

In [21]:
print(gen_wv_model)

Word2Vec<vocab=46908, vector_size=500, alpha=0.025>


In [22]:
# tokenizer = Tokenizer(46908)
# tokenizer.fit_on_texts(gen_words)
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 75)

model = keras.models.Sequential([

    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=500,
                            input_length=75,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    keras.layers.Conv1D(50, 3, activation='relu', padding='same', strides=1),
    keras.layers.MaxPool1D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(10),
    keras.layers.Activation('relu'),
    keras.layers.Dense(1),
    keras.layers.Activation('sigmoid')
])


y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y, random_state=42)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
118/118 [==============================] - 30s 236ms/step - loss: 0.4206 - accuracy: 0.8450 - precision_3: 0.8474 - recall_3: 0.9965 - val_loss: 0.4129 - val_accuracy: 0.8472 - val_precision_3: 0.8472 - val_recall_3: 1.0000
Epoch 2/10
118/118 [==============================] - 21s 181ms/step - loss: 0.3566 - accuracy: 0.8474 - precision_3: 0.8474 - recall_3: 1.0000 - val_loss: 0.4327 - val_accuracy: 0.8472 - val_precision_3: 0.8472 - val_recall_3: 1.0000
Epoch 3/10
118/118 [==============================] - 16s 140ms/step - loss: 0.2660 - accuracy: 0.8904 - precision_3: 0.8870 - recall_3: 0.9978 - val_loss: 0.4463 - val_accuracy: 0.8365 - val_precision_3: 0.8516 - val_recall_3: 0.9774
Epoch 4/10
118/118 [==============================] - 14s 118ms/step - loss: 0.1720 - accuracy: 0.9435 - precision_3: 0.9422 - recall_3: 0.9943 - val_loss: 0.5236 - val_accuracy: 0.8403 - val_precision_3: 0.8506 - val_recall_3: 0.9843
Epoch 5/10
118/118 [==============================] - 13s 11

In [23]:
val_recall = 0.8519 *100
val_precision = 0.9654 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

90.51056622461893

In [ ]:
!zip -r "/bilstm.zip" "/bert-with-intermediate-task.h"

  adding: bert-with-intermediate-task.h/ (stored 0%)
  adding: bert-with-intermediate-task.h/assets/ (stored 0%)
  adding: bert-with-intermediate-task.h/keras_metadata.pb (deflated 92%)
  adding: bert-with-intermediate-task.h/variables/ (stored 0%)
  adding: bert-with-intermediate-task.h/variables/variables.index (deflated 67%)
  adding: bert-with-intermediate-task.h/variables/variables.data-00000-of-00001 (deflated 48%)
  adding: bert-with-intermediate-task.h/fingerprint.pb (stored 0%)
  adding: bert-with-intermediate-task.h/saved_model.pb (deflated 90%)
